In [112]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

import tensorflow as tf
import keras

import warnings
warnings.filterwarnings('ignore')

In [113]:
#Importing the preprocessed data
tot_data = pd.read_csv("final_label_encoding.csv")
train_data=tot_data.iloc[0:18359]
test_data=tot_data.iloc[18359:]
ids=test_data['enrollee_id']
target=train_data['target']
train_data.drop('target',axis=1,inplace=True)
test_data.drop('target',axis=1,inplace=True)
train_data.drop('enrollee_id',axis=1,inplace=True)
test_data.drop('enrollee_id',axis=1,inplace=True)

In [114]:
#Normalizing the training hours column

x = train_data['training_hours'].reshape(-1, 1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
training_hours_scaled = x_scaled
del(train_data['training_hours'])
train_data['training_hours'] = training_hours_scaled

x = test_data['training_hours'].reshape(-1, 1)
x_scaled = min_max_scaler.fit_transform(x)
training_hours_scaled = x_scaled
del(test_data['training_hours'])
test_data['training_hours'] = training_hours_scaled

In [115]:
tf.reset_default_graph()

In [62]:
#Placeholders for input, target, loss penalty

inp = tf.placeholder(shape=(None,1,16),dtype=tf.float32)
tar = tf.placeholder(shape=(None),dtype=tf.float32)
lo = tf.placeholder(shape=(None),dtype=tf.float32)

In [63]:
#Defining the weight and bias variables

weight_1 = tf.Variable(tf.random_normal([64,30],mean=0,stddev=1))
weight_2 = tf.Variable(tf.random_normal([30,10],mean=0,stddev=1))
weight_3 = tf.Variable(tf.random_normal([10,1],mean=0,stddev=1))
bias_1 = tf.Variable(tf.random_normal([30],mean=0,stddev=1))
bias_2 = tf.Variable(tf.random_normal([10],mean=0,stddev=1))
bias_3 = tf.Variable(tf.random_normal([1],mean=0,stddev=1))

In [64]:
#LSTM layer with h(t) = 64

lstm = keras.layers.LSTM(64,input_shape=[None,1,16])(inp)

In [65]:
#Dense layer with 30 neurons

layer_1 =tf.nn.relu(tf.add(tf.matmul(lstm,weight_1),bias_1))

In [66]:
#Dropout layer for regularization

dropout1 = keras.layers.Dropout(0.2)(layer_1)

In [67]:
#Dense layer with 10 neurons

layer_2 = tf.nn.relu(tf.add(tf.matmul(dropout1,weight_2),bias_2))

In [68]:
dropout2 = keras.layers.Dropout(0.2)(layer_2)

In [69]:
#Dense layer with 1 neuron

layer_3 = tf.add(tf.matmul(dropout2,weight_3),bias_3)

In [70]:
#Sigmoid layer for finding the AUC after every epoch

sig_layer_3 = tf.nn.sigmoid(layer_3)

In [71]:
#Binary cross entropy loss with L2 regularization

los = (tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=layer_3,labels=tar)))+ (0.005*tf.nn.l2_loss(weight_1)) + (0.005*tf.nn.l2_loss(weight_2)) + (0.005*tf.nn.l2_loss(weight_3)) 

In [72]:
#Adam optimizer with default learning rate

opt = tf.train.AdamOptimizer()

In [73]:
#Optimizing loss with additional penalty for loss with minor class 1

model = opt.minimize(lo * los)

In [74]:
#Initializing the variables

init = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())

In [75]:
#Creating the tf session

ses = tf.InteractiveSession()
ses.run(init)

In [76]:
from sklearn.metrics import roc_auc_score

In [77]:
#Reshaping training data to 3-d to fit LSTM input

train_data = np.array(train_data)
train_data = train_data.reshape([18359,1,16])

In [78]:
#Reshaping the test data for LSTM input

test_data = np.array(test_data)
test_data = test_data.reshape([15021,1,16])

In [110]:
#Using online learning and with loss multipled by 200 when the data point is with label 1 and then pring the AUC after every epoch
for epoch in range(0,50):
    for i in range(0,(len(train_data))):
        
        a = train_data[i].reshape([1,1,16])
        if(target[i] == 0):
            c = 1.0
        else:
            c = 200.0

        ses.run(model,feed_dict={inp:a,tar:target[i],lo:c})
        
    result = ses.run([sig_layer_3],feed_dict={inp:train_data})
    result = result[0]
    result = np.array(result)
    result = result.reshape([18359,1])
    print(roc_auc_score(y_true=target,y_score=result))    
    
    print("\n")

0.681664352568


0.681906925345


0.68191498695


0.682047453478


0.682016513997


0.68198846013


0.682007572474


0.682031537308


0.681996948754


0.682094749087




In [111]:
#Predicting on test data and saving to file

predictions = ses.run([sig_layer_3],feed_dict={inp:test_data})
predictions=predictions[0]
predictions=predictions.reshape([15021])

sub=pd.DataFrame({'enrollee_id':ids,'target':predictions})
sub.to_csv("predictions_lstm.csv",index=False)